In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install google-api-python-client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import datetime

In [4]:
# If modifying these scopes, delete the file token_sheets.json.
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# https://docs.google.com/spreadsheets/d/1is8ZmX5mnwDU0YESiHi_yRqhysTJRF0UePIVNkzAOhk/edit#gid=487175113 sheets do cefacil

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1is8ZmX5mnwDU0YESiHi_yRqhysTJRF0UePIVNkzAOhk'
SAMPLE_RANGE_NAME = 'Reservas autorizadas!A:H'


creds_sheets = None
# The file token_sheets.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token_sheets.json'):
    creds_sheets = Credentials.from_authorized_user_file('token_sheets.json', SCOPES_SHEETS)
# If there are no (valid) credentials available, let the user log in.
if not creds_sheets or not creds_sheets.valid:
    if creds_sheets and creds_sheets.expired and creds_sheets.refresh_token:
        creds_sheets.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES_SHEETS)
        creds_sheets = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token_sheets.json', 'w') as token:
        token.write(creds_sheets.to_json())

In [5]:
SCOPES_CALENDAR = ['https://www.googleapis.com/auth/calendar']

creds_calendar = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token_calendar.json'):
    creds_calendar = Credentials.from_authorized_user_file('token_calendar.json', SCOPES_CALENDAR)
# If there are no (valid) credentials available, let the user log in.
if not creds_calendar or not creds_calendar.valid:
    if creds_calendar and creds_calendar.expired and creds_calendar.refresh_token:
        creds_calendar.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES_CALENDAR)
        creds_calendar = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token_calendar.json', 'w') as token:
        token.write(creds_calendar.to_json())

In [6]:
def get_events_from_sheets(spreadsheet_id, range_name, creds):
    # Constrói o serviço Sheets usando as credenciais fornecidas
    service = build('sheets', 'v4', credentials=creds)

    # Faz a solicitação para obter os valores da planilha
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id,range=range_name,majorDimension='ROWS').execute()

    events = []  # Lista para armazenar os eventos

    # Itera sobre os valores retornados
    for i in result['values']:
        if not i == []:
            events.append(i)  # Adiciona a linha (evento) à lista de eventos

    return events[1:]


In [7]:
def show_events_sheet(spreadsheet_id, range_name, creds):
    # Obtém os eventos da planilha usando a função get_events_from_sheets
    eventos = get_events_from_sheets(spreadsheet_id, range_name, creds)

    # Itera sobre os eventos
    for i in eventos:
        # Imprime o evento
        print(i)

        # Obtém a última linha
        last_row = i

        # Extrai os valores das colunas
        local = last_row[0]
        start_date = last_row[1]
        start_time = last_row[2]
        end_time = last_row[3]
        name = last_row[4]
        n_usp = last_row[5]
        email = last_row[6]
        description = last_row[7]

        # Imprime os valores
        print("Date:", start_date)
        print("Time:", start_time)
        print("Name:", name)
        print("ID Number:", n_usp)
        print("Email:", email)
        print("Local:", local)
        print("Start Date:", start_date)
        print("Start Time:", start_time)
        print("End Time:", end_time)
        print("Description:", description)
        print()


In [8]:
def week_events(creds, calendar_id):
    try:
        # Constrói o serviço Calendar usando as credenciais fornecidas
        service = build('calendar', 'v3', credentials=creds)

        # Obtém a data e hora atuais no formato ISO e acrescenta uma semana para obter a data e hora da próxima semana
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indica tempo em UTC
        one_week_later = (datetime.datetime.utcnow() + datetime.timedelta(weeks=1)).isoformat() + 'Z'

        # Faz a chamada para a API do Calendar para obter os eventos da próxima semana
        events_result = service.events().list(
            calendarId=calendar_id,
            timeMin=now,
            timeMax=one_week_later,
            singleEvents=True,
            orderBy='startTime'
        ).execute()

        events = events_result.get('items', [])  # Obtém os eventos da resposta

        if not events:
            print('No upcoming events found.')
            return []
        else:
            return events

    except HttpError as error:
        print('An error occurred: %s' % error)


In [9]:
def create_event(id_agenda, titulo, descricao, local, data_inicio, hora_inicio, data_fim, hora_fim, creds, convidados=None):
    # Constrói o serviço Calendar usando as credenciais fornecidas
    service = build('calendar', 'v3', credentials=creds)
    
    try:
        # Cria um dicionário com os detalhes do evento
        evento = {
            'summary': titulo,  # Título do evento
            'description': descricao,  # Descrição do evento
            'location': local,  # Local do evento
            'start': {
                'dateTime': f'{data_inicio}T{hora_inicio}',  # Data e hora de início do evento
                'timeZone': 'America/Sao_Paulo',  # Substitua pelo fuso horário correto
            },
            'end': {
                'dateTime': f'{data_fim}T{hora_fim}',  # Data e hora de término do evento
                'timeZone': 'America/Sao_Paulo',  # Substitua pelo fuso horário correto
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'email', 'minutes': 60}  # Configuração do lembrete por email, 60 minutos antes do evento
                ],
            },
            'guestsCanInviteOthers': True,  # Permite que os convidados convidem outras pessoas
            'guestsCanSeeOtherGuests': True,  # Permite que os convidados vejam outros convidados
        }

        if convidados is not None:
            # Se houver convidados especificados, adiciona-os ao evento
            evento['attendees'] = [{'email': email} for email in convidados]

        # Insere o evento no calendário usando o serviço
        evento_criado = service.events().insert(calendarId=id_agenda, body=evento).execute()

        # Imprime os detalhes do evento criado
        print("Evento criado:")
        print(f"Título: {evento_criado['summary']}")
        print(f"Início: {evento_criado['start']['dateTime']}")
        print(f"Fim: {evento_criado['end']['dateTime']}")

    except HttpError as error:
        print('An error occurred: %s' % error)


In [10]:
def convert_date_format(date_string):
    # Esta função converte uma string de data no formato "dd/mm/yyyy" para o formato "yyyy-mm-dd"
    day, month, year = date_string.split('/')
    new_date_string = f"{year}-{month}-{day}"
    return new_date_string

def convert_event_sheet_to_event_calendar(event):
    # Esta função converte um evento da planilha para um formato compatível com o calendário

    local = event[0]  # Obtém o local do evento do índice 0 da lista event
    start_date = convert_date_format(event[1])  # Converte a data de início do evento para o formato correto
    start_time = event[2] + '-03:00'  # Obtém a hora de início do evento e adiciona o fuso horário correto (-03:00)
    end_time = event[3] + '-03:00'  # Obtém a hora de término do evento e adiciona o fuso horário correto (-03:00)
    name = event[4]  # Obtém o nome do evento
    n_usp = event[5]  # Obtém o número USP
    email = event[6]  # Obtém o email
    description = event[7]  # Obtém a descrição do evento

    # Retorna os valores convertidos em uma ordem específica (local, start_date, start_time, end_time, name, n_usp, email, description)
    return local, start_date, start_time, end_time, name, n_usp, email, description


In [11]:
def create_week_events(spreadsheet_id, range_name, creds_s, id_agenda, creds_c):
    # Esta função cria eventos da semana no calendário com base nos eventos da planilha

    # Obtém a lista de eventos da planilha
    events_from_sheets_list = get_events_from_sheets(spreadsheet_id, range_name, creds_s)
    # Obtém os eventos da semana do calendário
    events_from_calendar = week_events(creds_c, id_agenda)

    # Itera sobre os eventos da planilha
    for event_sheet in events_from_sheets_list:
        # Converte o evento da planilha para um formato compatível com o calendário
        local, start_date, start_time, end_time, name, n_usp, email, description = convert_event_sheet_to_event_calendar(event_sheet)
        titulo = 'Reserva ' + local

        found = False

        # Verifica se o evento já existe na agenda do calendário
        if len(events_from_calendar) > 0:
            for event_calendar in events_from_calendar:
                if event_calendar['start']['dateTime'] == f'{start_date}T{start_time}' and \
                    event_calendar['end']['dateTime'] == f'{start_date}T{end_time}' and \
                        event_calendar['location'] == local:
                    found = True
                    break

        # Se o evento não foi encontrado, cria um novo evento no calendário
        if not found:
            create_event(id_agenda, titulo, description, local, start_date, start_time, start_date, end_time, creds_calendar, [email])

# Chamada da função create_week_events com argumentos de exemplo
create_week_events(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, creds_sheets, 'testecefacil@gmail.com', creds_calendar)


Evento criado:
Título: Reserva Piscina
Início: 2023-06-14T12:38:00-03:00
Fim: 2023-06-14T13:28:00-03:00
